<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
!pip install folium

In [125]:
import folium
lat = 25.027472815739536
lng = 121.52993850781064
m = folium.Map(location=(lat, lng), zoom_start=15)

In [126]:
# ==============================================================================
# 0. 環境設置與函式庫導入
# ==============================================================================
# --- 運行環境設定（請在 Colab Cell 中執行）---
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
              gradio pandas beautifulsoup4 google-generativeai python-dateutil scikit-learn

In [127]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [128]:
# 認證與 Gemini 配置（請在 Colab Cell 中執行）
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

from google.colab import userdata
# 從 Colab Secrets 中獲取 API 金鑰
try:
    api_key = userdata.get('HW3')
    # 確保金鑰存在，否則 genai.configure 會報錯
    if not api_key:
        raise ValueError("Colab Secret 'HW3' is empty or not found.")

    genai.configure(api_key=api_key)
    print("✅ Gemini API Key 配置成功。")
except Exception as e:
    print(f"⚠️ Gemini API Key 配置失敗，請檢查 Colab Secrets 中的 'HW3'：{e}")

✅ Gemini API Key 配置成功。


In [129]:
# ==============================================================================
# 1. 全域變數與 Sheet/DataFrame 設置
# ==============================================================================

# 請確保這個 Sheet URL 存在且您有編輯權限
SHEET_URL = "https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing"
WORKSHEET_NAME = "food_choice"
TIMEZONE = "Asia/Taipei"

In [130]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_restaurants.head()

,店家名稱,類別/招牌品項,參考地點,價位,距離,營業,備註/特色,lat,lng
0,難吃,便當菜/快餐,師大,$$,長,是,牛肉麵,25.02207668,121.5283606
1,溏老鴨平價小火鍋,小火鍋,新生南路,$$,長,是,平實的餐廳，專售海鮮、肉類和蔬菜火鍋料理，並供應家常甜點。,25.01981157,121.5334397
2,大埔鐵板燒,日式料理,公館,$$,短,是,沒有免費無限加菜了,25.01239099,121.5355401
3,SUKIYA,日式料理,公館,$,長,是,快速出餐,25.01767512,121.5318739
4,鍋 in,小火鍋,公館,$$,短,否,人一直都很多的地方,25.012641,121.535253


In [131]:
for i in range(0, len(df_restaurants)):
  lat = df_restaurants['lat'][i]
  lng = df_restaurants['lng'][i]
  tooltip_str = df_restaurants['店家名稱'][i]
  popup_str = df_restaurants['類別/招牌品項'][i]

  folium.Marker(
      location = [lat, lng],
      tooltip = tooltip_str,
      popup = popup_str,
      icon=folium.Icon(icon="cloud"),
  ).add_to(m)


In [132]:
m

In [133]:
import pandas as pd
import numpy as np
import random
import gradio as gr
import asyncio
import json

In [134]:
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""

    # 轉換篩選條件為列表
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    # Gradio CheckboxGroup 回傳列表，但單一勾選時需處理
    available_status = ['是'] if is_available else ['是', '否']

    # 應用篩選條件
    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    # 隨機抽取 3 家，不足 3 家則全數取出
    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    # 準備推薦清單文字
    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    # 準備 AI 摘要的輸入字串
    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input

In [135]:
async def generate_summary(ai_input):
    """呼叫 LLM 產生決策建議摘要 (使用 gemini-2.5-pro)"""
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    # Fallback if model configuration failed
    await asyncio.sleep(1) # 模擬網路延遲
    mock_summary = f"""
    **AI 助理 (模擬):** 今天的選擇是：{'、'.join([r.split('(')[0].strip() for r in ai_input.split('、')])}。
    如果您追求**效率和銅板價**，{ai_input.split('、')[0].split('，')[0]} (價位{ai_input.split('、')[0].split('，')[1].split('價位')[1]}) 肯定是首選，快速又飽足。
    若想來點**儀式感和高品質**，{'、'.join(ai_input.split('、')[1:])} 提供了更好的用餐體驗，但距離和花費會稍高一點。
    決定權在你！
    (請確保您已安裝 'google-genai' 並在 Colab Secrets 中設置 'gemini' API key 以啟用真實 AI 推薦)
    """
    return mock_summary.strip()


    system_prompt = (
        "您是一位幽默且極具效率的午餐決策顧問。您的任務是根據提供的餐廳清單，"
        "分析其優缺點（如價格、距離、特色）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以輕鬆口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要，不超過 150 字。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    user_query = f"這是三家推薦的餐廳清單：{ai_input}。請綜合考量其價格、距離和特色，為我提供一份快速決策建議。"

    try:
        # 使用 asyncio.to_thread 讓同步的 SDK 呼叫不會阻塞 Gradio 的事件迴圈
        response = await asyncio.to_thread(
            model.generate_content,
            contents=user_query,
            config=genai.types.GenerateContentConfig(
                system_instruction=system_prompt,
                temperature=0.7,
            )
        )
        return response.text.strip()

    except Exception as e:
        error_msg = f"呼叫 Gemini API 失敗，請檢查 API 金鑰或配額：{e}。"
        return error_msg


In [136]:
# 介面更新函數
async def wrapper_recommend(price_filter, distance_filter, is_available):
    """Gradio 主要流程：篩選 -> 推薦 -> 呼叫 AI 摘要"""

    # 1. 篩選與推薦
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_restaurants, price_filter, distance_filter, is_available
    )

    # 2. 呼叫 AI 摘要
    ai_summary = await generate_summary(ai_input)

    # 3. 準備給 Gradio 輸出
    # 將推薦結果轉換為 Dataframe (僅顯示名稱、價位、距離)
    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    # 處理空結果
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）"

    return display_df, recommend_list, ai_summary

In [137]:
# 設定 Gradio 介面元件
with gr.Blocks(title="迷你黑客松午餐決策器") as demo:
    gr.Markdown(
        """
        # 🍽️ 迷你黑客松午餐/聚會決策器
        ### 目標：從餐廳清單中篩選，並隨機推薦 3 家，最後由 AI 助您快速決策！
        **🚨 注意：請確保您已在 Colab 中讀取 Google Sheet 數據，否則將使用模擬數據。**
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 條件篩選")
            price_select = gr.CheckboxGroup(
                ['$', '$$', '$$$'],
                label="價位過濾 (可複選)",
                value=['$', '$$'],
                info="(銅板內$$: 300+)"
            )
            distance_select = gr.CheckboxGroup(
                ['短', '中', '長'],
                label="距離過濾 (可複選)",
                value=['短', '中'],
                info="(短: 步行5分, 中: 步行10分, 長: 需騎車)"
            )
            available_toggle = gr.Checkbox(
                label="只看今日營業",
                value=True
            )
            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=2):
            gr.Markdown("## 推薦清單 (隨機 3 家)")
            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "距離"],
                datatype=["str", "str", "str"],
                label="隨機推薦結果"
            )
            output_list = gr.Textbox(label="推薦清單細節")

    gr.Markdown("## 🤖 AI 決策建議 (約 100 字)")
    output_summary = gr.Textbox(
        label="AI 決策顧問報告",
        interactive=False,
        info="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。"
    )

    # 定義點擊按鈕後的行為 (使用異步包裝函數)
    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[
            price_select,
            distance_select,
            available_toggle
        ],
        outputs=[output_dataframe, output_list, output_summary]
    )

    gr.Markdown(
        """
        ---
        **[開發者備註]** AI 摘要已設定為使用 `gemini-2.5-pro` 呼叫。請確保您已執行以下步驟：
        1.  在 Colab 中安裝 `google-genai` 等必要套件。
        2.  在 Colab Secrets 中設定名為 `gemini` 的 API 金鑰。
        """
    )



In [138]:
# 運行 Gradio 應用
if __name__ == "__main__":
    # 在 Colab 或一般環境中運行
    print("Gradio 應用正在啟動...")
    demo.launch(inbrowser=True)

Gradio 應用正在啟動...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a2532b4d0a99ad20ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
